In [ ]:
from scrapper import get_links_and_save_content
from urllib.parse import urljoin, urlparse
import json
from queue import Queue
import pickle
from collections import defaultdict 

web_content_directory = 'WebContent3000'
seed_url = '<give the seed URL>'
pages_to_crawl = 3000
domain = '<give the domain name>'
web_graph = 'WebGraph3000'
visited_url = 'VisitedUrl3000'
visiting_url = 'VisitingUrl3000'

class WebCrawler:
    
    def __init__(self,base_url,domain,pages_to_crawl=3000):
        
        self.visiting_url=[]
        self.visited_url=set([])
        self.pages_to_crawl = pages_to_crawl
        self.visiting_url.append(base_url)
        self.domain=domain
        self.web_content_directory = web_content_directory
        self.page_links_dict = defaultdict(list)
        
    def parse_url(self,base_url,links):
        for link in links:
            url = link['href']
            if url != None and (url.startswith('/') or self.domain in url) and self. is_valid_extension(url) and '@' not in url:
                url = urljoin(base_url,url)
                if url[-1]=='/':
                    url= url[:-1]
                if 'https' not in url:
                    url = url.replace("http", "https")
                self.page_links_dict[base_url].append(url)
                if url not in self.visited_url and url not in self.visiting_url:
                    self.visiting_url.append(url)
                    print(url)        
        
    def start_crawling(self):
        
        crawled_page_count = 0
        
        while crawled_page_count < self.pages_to_crawl and len(self.visiting_url)>0:
            url = self.visiting_url[0]
            self.visiting_url=self.visiting_url[1:]
            self.visited_url.add(url)
            filename = self.web_content_directory + '/' + str(crawled_page_count)+'.txt'
            try:
                success,links = get_links_and_save_content(url,filename)
                
                print('-----------------links form {} this url----------------'.format(url))
                if success:
                    self.parse_url(url,links)
                    crawled_page_count += 1
                print('-----------------------------links ended------------------------------')
                    
            except Exception as e:
                print(e)
                with open(web_graph + '.json', 'w+') as fd:
                    json.dump(self.page_links_dict, fd)

                with open(visited_url, 'wb') as fv:
                    pickle.dump(self.visited_url, fv)
                    
                with open(visiting_url, 'wb') as fvl:
                    pickle.dump(self.visiting_url,fvl)
                
                
        with open(web_graph +'.json', 'w+') as fd:
                json.dump(self.page_links_dict, fd)

        with open(visited_url, 'wb') as fv:
                pickle.dump(self.visited_url, fv)
                    
        with open(visiting_url, 'wb') as fvl:
                pickle.dump(self.visiting_url,fvl)
                
    def is_valid_extension(self,url):
        invalid_extensions = ["pdf", "jpg", "jpeg", "doc", "docx", "ppt", "pptx", "png", "txt", "exe", "ps", "psb"]
        if True in [ext in url for ext in invalid_extensions]:
            return False
        return True
            
            
WebCrawler(seed_url,domain,pages_to_crawl,3000).start_crawling()